In [1]:
import numpy as np
import pandas as pd
import os
import shutil
import matplotlib.pyplot as plt
import tensorflow
from tensorflow import keras
from keras.models import Sequential,load_model,Model
from keras.layers import Conv2D,MaxPool2D,AveragePooling2D,Dense,Flatten,ZeroPadding2D,BatchNormalization,Activation,Add,Input,Dropout,GlobalAveragePooling2D
from keras.optimizers import SGD
from keras.initializers import glorot_uniform
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint,EarlyStopping,ReduceLROnPlateau
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!unzip /content/drive/MyDrive/archive.zip > /dev/null

In [5]:
data_dir = '/content/Dataset'
train_dir = data_dir + '/Train'
valid_dir = data_dir + '/Validation'
test_dir = '/content/Dataset/Test'
diseases = os.listdir(train_dir)
print("Total disease classes are: {}".format(len(diseases)))

Total disease classes are: 2


In [6]:
from tensorflow.keras.applications.efficientnet_v2 import EfficientNetV2S
base_model_tf=ResNet50(include_top=False,weights='imagenet',input_shape=(224,224,3),classes=43)

94765736/94765736 [==============================] - 5s 0us/step


In [7]:

base_model_tf.trainable=False

pt=Input(shape=(224,224,3))
func=tensorflow.cast(pt,tensorflow.float32)
x=preprocess_input(func)
model_resnet=base_model_tf(x,training=False)
model_resnet=GlobalAveragePooling2D()(model_resnet)
model_resnet=Flatten()(model_resnet)

model_resnet=Dense(128,activation='relu')(model_resnet)
model_resnet=BatchNormalization()(model_resnet)
model_resnet=Dense(64,activation='relu')(model_resnet)
model_resnet=BatchNormalization()(model_resnet)
model_resnet=Dense(2,activation='softmax')(model_resnet)

model_main=Model(inputs=pt,outputs=model_resnet)
model_main.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 tf.cast (TFOpLambda)        (None, 224, 224, 3)       0         
                                                                 
 tf.__operators__.getitem (  (None, 224, 224, 3)       0         
 SlicingOpLambda)                                                
                                                                 
 tf.nn.bias_add (TFOpLambda  (None, 224, 224, 3)       0         
 )                                                               
                                                                 
 resnet50 (Functional)       (None, 7, 7, 2048)        23587712  
                                                                 
 global_average_pooling2d (  (None, 2048)              0     

In [8]:
train_datagen= ImageDataGenerator(shear_range=0.2,zoom_range=0.2,horizontal_flip=False,vertical_flip=False
                                  ,fill_mode='nearest',width_shift_range=0.2,height_shift_range=0.2)

val_datagen=ImageDataGenerator()

train= train_datagen.flow_from_directory(directory=train_dir,batch_size=32,target_size=(224,224),
                                         color_mode='rgb',class_mode='categorical',seed=42)

valid=val_datagen.flow_from_directory(directory=valid_dir,batch_size=32,target_size=(224,224),color_mode='rgb',class_mode='categorical',seed=42)

Found 140002 images belonging to 2 classes.
Found 39428 images belonging to 2 classes.


In [9]:
es = EarlyStopping(monitor='val_accuracy',verbose=1,patience=7,mode='auto')
mc = ModelCheckpoint(filepath='/content/cnn3',monitor='val_accuracy',verbose=1,save_best_only=True)
lr = ReduceLROnPlateau(monitor='val_accuracy',verbose=1,patience=5,min_lr=0.001)

In [10]:
model_main.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model_main.fit(train,validation_data=valid,epochs=30,steps_per_epoch=200,verbose=1,callbacks=[mc,es,lr])

Epoch 1/30
200/200 [==============================] - ETA: 0s - loss: 0.5675 - accuracy: 0.7211
Epoch 1: val_accuracy improved from -inf to 0.77896, saving model to /content/cnn3
200/200 [==============================] - 272s 1s/step - loss: 0.5675 - accuracy: 0.7211 - val_loss: 0.4825 - val_accuracy: 0.7790 - lr: 0.0010
Epoch 2/30
200/200 [==============================] - ETA: 0s - loss: 0.4890 - accuracy: 0.7591
Epoch 2: val_accuracy improved from 0.77896 to 0.78627, saving model to /content/cnn3
200/200 [==============================] - 244s 1s/step - loss: 0.4890 - accuracy: 0.7591 - val_loss: 0.4744 - val_accuracy: 0.7863 - lr: 0.0010
Epoch 3/30
200/200 [==============================] - ETA: 0s - loss: 0.4625 - accuracy: 0.7763
Epoch 3: val_accuracy did not improve from 0.78627
200/200 [==============================] - 228s 1s/step - loss: 0.4625 - accuracy: 0.7763 - val_loss: 0.4735 - val_accuracy: 0.7768 - lr: 0.0010
Epoch 4/30
200/200 [==============================] - ETA

In [ ]:
model_main.save("resnetttt.h5")

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
